In [29]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import torch
from torch import nn
import torch.nn.functional as F
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from torch.utils.data import Dataset, DataLoader
import torchvision
import torchvision.transforms as transforms
import pandas as pd
from brainflow.data_filter import DataFilter, FilterTypes, DetrendOperations, WindowOperations
from sklearn.metrics import f1_score
import torch.optim

In [30]:
class ToTensor(object):
    def __call__(self, sample):
        window, labels = sample['window'], sample['labels']
        return {'window': torch.tensor(window.values).to(torch.float32), 
                'labels': torch.tensor(labels).to(torch.float32)}#.values)}

class AbsoluteValue(object):
    def __call__(self, sample):
        window, labels = sample['window'], sample['labels']
        return {'window': torch.abs(window), 
                'labels': labels}

class MinNormalize(object):
    def __call__(self, sample):
        window, labels = sample['window'], sample['labels']
        for channel in range(len(window)):
            #print(len(window))
            min = torch.min(window[channel])
            window[channel] = window[channel] - min
            window[channel] = window[channel] / torch.max(window[channel])
            window[channel] = torch.nan_to_num(window[channel])
        return {'window': (window.unsqueeze(0)), 
                'labels': labels}

# class FixLabels(object):
#     def __call__(self, sample):
#         window, labels = sample['window'], sample['labels']
#         if labels[0] == 2:
#             labels = labels - 1
#         return {'window': window, 
#                 'labels': labels}

class EEGDataset(Dataset):
    def __init__(self, csv_file, transform=None):
        self.csv_file = pd.read_csv(csv_file, delimiter='\t', header=None)
        self.transform = transform
    
    def __len__(self):
        return len(self.csv_file)
    
    def __getitem__(self, idx):
        # if idx >= len(self.csv_file) / 625:
        #     return

        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        sample_size = 625
        
        # window = self.csv_file.iloc[idx*625:idx*625+625, 1:17].transpose()
        window = self.csv_file.iloc[idx*sample_size:idx*sample_size+sample_size, 1:17].transpose()
        labels = [self.csv_file.iloc[idx*sample_size, 32]]#:idx*625+625, 32] # remove brackets for timestep prediction
        sample = {'window': window, 'labels': labels}

        if self.transform:
            sample = self.transform(sample)
        
        return sample

In [31]:
eeg_dataset = EEGDataset(csv_file='./randomRBFull.csv', transform=transforms.Compose([ToTensor(),
                                                                                  AbsoluteValue(),
                                                                                  MinNormalize()]))

# eeg_dataset = EEGDataset(csv_file='./fullData.csv', transform=transforms.Compose([ToTensor(),
#                                                                                   AbsoluteValue()]))

#eeg_dataset = EEGDataset(csv_file='./fullData.csv', transform=transforms.Compose([ToTensor()]))

eeg_dataset = torch.utils.data.Subset(eeg_dataset, range(0, 205))

print(eeg_dataset[0]['window'].shape)

#print(eeg_dataset[0]['window'][0][1])

# for channel in range(len(eeg_dataset[0]['window'][0])):
#     print(torch.min(eeg_dataset[0]['window'][0][channel]))

print("Full max:", torch.max(eeg_dataset[0]['window'][0]))
print("Full min:", torch.min(eeg_dataset[0]['window'][0]))

torch.manual_seed(50) # 80
train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(eeg_dataset, [0.7, 0.2, 0.1])

# train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
# val_dataloader = DataLoader(val_dataset, batch_size=16, shuffle=True)
# test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=True)

print(len(train_dataset), len(val_dataset), len(test_dataset))

trainreds = 0
trainblues = 0

for i in range(len(train_dataset)):
    if train_dataset[i]['labels'][0] == 1:
        trainblues += 1
    else:
        trainreds += 1

print("Class imbalance:", trainblues, trainreds)

zerocount = 0
onecount = 0

for i in range(len(val_dataset)):
    if val_dataset[i]['labels'][0] == 1:
        onecount += 1
    else:
        zerocount += 1

print(onecount, zerocount)

zerocount = 0
onecount = 0

for i in range(len(test_dataset)):
    if test_dataset[i]['labels'][0] == 1:
        onecount += 1
    else:
        zerocount += 1

print(onecount, zerocount)

print(train_dataset[7])

torch.Size([1, 16, 625])
Full max: tensor(1.)
Full min: tensor(0.)
144 41 20
Class imbalance: 65 79
26 15
11 9
{'window': tensor([[[0.0762, 0.6587, 0.0222,  ..., 0.8704, 0.3898, 0.7919],
         [0.3685, 0.5900, 0.3406,  ..., 0.7320, 0.7171, 0.6550],
         [0.6970, 0.5925, 0.6846,  ..., 0.3982, 0.1917, 0.4885],
         ...,
         [0.3167, 0.5005, 0.3612,  ..., 0.5493, 0.7566, 0.4862],
         [0.1919, 0.3245, 0.2231,  ..., 0.6642, 0.8548, 0.5869],
         [0.4250, 0.3126, 0.4274,  ..., 0.6162, 0.8771, 0.6196]]]), 'labels': tensor([1.])}


In [32]:
print(test_dataset[0]['window'].shape)

torch.Size([1, 16, 625])


In [33]:
num_train = len(train_dataset)
window_tensors = torch.empty(size=(num_train, 1, 16, 625))
label_tensors = torch.empty(size=(num_train, 1))

for i in range(num_train):
    window_tensors[i] = train_dataset[i]['window']
    label_tensors[i] = train_dataset[i]['labels']

torch.save(window_tensors, 'window_tensors.pt')
torch.save(label_tensors, 'label_tensors.pt')

window_tensors = torch.load('window_tensors.pt')
label_tensors = torch.load('label_tensors.pt')

print(window_tensors.shape)
print(label_tensors.shape)

torch.Size([144, 1, 16, 625])
torch.Size([144, 1])


/var/folders/jy/pdjxhl6d6q58q0b7w0sk_g_80000gn/T/ipykernel_58358/3990837986.py:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  window_tensors = torch.load('window_tensors.

In [34]:
num_val = len(val_dataset)
val_window_tensors = torch.empty(size=(num_val, 1, 16, 625))
val_label_tensors = torch.empty(size=(num_val, 1))

for i in range(num_val):
    val_window_tensors[i] = val_dataset[i]['window']
    val_label_tensors[i] = val_dataset[i]['labels']

torch.save(val_window_tensors, 'val_window_tensors.pt')
torch.save(val_label_tensors, 'val_label_tensors.pt')

val_window_tensors = torch.load('val_window_tensors.pt')
val_label_tensors = torch.load('val_label_tensors.pt')

print(val_window_tensors.shape)
print(val_label_tensors.shape)

torch.Size([41, 1, 16, 625])
torch.Size([41, 1])


/var/folders/jy/pdjxhl6d6q58q0b7w0sk_g_80000gn/T/ipykernel_58358/2744465643.py:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  val_window_tensors = torch.load('val_window_

In [ ]:
class EEGRedBlueClassifier(nn.Module):
    def __init__(self):
        super(EEGRedBlueClassifier, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, (8, 5), stride=(1, 5))
        self.conv2 = nn.Conv1d(32, 64, 5, stride=5)
        self.pool1 = nn.AdaptiveAvgPool1d(1)
        self.fc1 = nn.Linear(64, 32)
        self.fc2 = nn.Linear(32, 1)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = x.squeeze()
        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool1(x)
        #print("Presqueeze:", x)
        x = x.squeeze()
        #print("Postsqueeze", x)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.sigmoid(x)
        return x
    
    # def forward(self, x):
    #     print(x.shape)
    #     x = self.conv1(x)
    #     print(x.shape)
    #     x = F.relu(x)
    #     x = x.squeeze()
    #     x = self.conv2(x)
    #     print(x.shape)
    #     x = F.relu(x)
    #     x = self.pool1(x)
    #     print(x.shape)
    #     #print("Presqueeze:", x)
    #     x = x.squeeze()
    #     #print("Postsqueeze", x)
    #     print(x.shape)
    #     x = self.fc1(x)
    #     print(x.shape)
    #     x = F.relu(x)
    #     print(x.shape)
    #     x = self.fc2(x)
    #     print(x.shape)
    #     x = F.sigmoid(x)
    #     return x


model = EEGRedBlueClassifier()
# model(torch.randn((1250,)).unsqueeze(0))

optimizer = torch.optim.Adam(model.parameters())
# scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, "min")
# classcounts = torch.tensor([trainreds, trainblues])
# maxclass = torch.max(classcounts)
# classweights = classcounts/maxclass
# criterion = nn.BCEWithLogitsLoss(weight=classweights)
criterion = nn.BCELoss()

In [36]:
losses = []
lossi = []
vallosses = []
vallossi = []

In [37]:
minValLoss = 9999

for epoch in tqdm(range(2500)):
    for i in range(len(window_tensors)):
        optimizer.zero_grad()
        model.train()
        y_pred = model(window_tensors[i])
        # print(y_pred)

        # if epoch == 0:
        #     print(y_pred)

        loss = criterion(y_pred, label_tensors[i])
        lossi.append(loss.item())
        loss.backward()
        optimizer.step()

        model.eval()
        for i in range(len(val_window_tensors)):
            val_pred = model(val_window_tensors[i])
            loss = criterion(val_pred, val_label_tensors[i])
            vallossi.append(loss.item())
        
        # if epoch <= 1:
        #     for p in model.parameters():
        #         print(p.grad)
    
    losses.append(np.mean(lossi))
    avgvalloss = np.mean(vallossi)
    if avgvalloss < minValLoss:
        minValLoss = avgvalloss
        torch.save(model.state_dict(), 'bestvalmodel.pt')
    vallosses.append(avgvalloss)
    if epoch % 5 == 0:
        plt.plot(losses, "b")
        plt.plot(vallosses, "g")
        plt.savefig('./valloss.png')
        plt.close()
    lossi = []
    vallossi = []
    

plt.plot(losses, "b")
plt.plot(vallosses, "g")

print(losses)

print(losses[-1])
torch.save(model.state_dict(), 'finaltrainmodel.pt')

  0%|          | 0/2500 [00:00<?, ?it/s]


ValueError: Target size (torch.Size([1])) must be the same as input size (torch.Size([2]))

In [ ]:
minindex = torch.argmin(torch.tensor(vallosses))
maxindex = torch.argmax(torch.tensor(vallosses))
print(losses[minindex], vallosses[minindex])
print(vallosses[maxindex], vallosses[maxindex] - vallosses[minindex])

In [ ]:
modelWeights = torch.load('./bestvalmodel.pt', weights_only=True)
#modelWeights = torch.load('./finaltrainmodel.pt', weights_only=True)
model = EEGRedBlueClassifier()
model.load_state_dict(modelWeights)
model.eval()
truepreds = 0
falsepreds = 0
truepositives = 0
falsepositives = 0
truenegatives = 0
falsenegatives = 0
preds = []
labels = []

with torch.no_grad():
    for i in range(len(test_dataset)):
        if i < 3:
            print(test_dataset[i]['window'])
        y_pred = model(test_dataset[i]['window'])
        # print(y_pred, test_dataset[i]['labels'])
        pred = (y_pred >= 0.5)
        preds.append(pred)
        label = test_dataset[i]['labels']
        labels.append(label)
        if pred == label:
            truepreds += 1
            if pred == 1:
                truepositives += 1
            else:
                truenegatives += 1
        else:
            falsepreds += 1
            if pred == 1:
                falsepositives += 1
            else:
                falsenegatives += 1

print(truepreds / (truepreds + falsepreds))
print(truepositives, truenegatives, falsepositives, falsenegatives)

f1Score = 2 * truepositives / (2 * truepositives + falsepositives + falsenegatives)
print(f1Score)

f1ScoreSKL = f1_score(labels, preds)
print(f1ScoreSKL)

In [ ]:
print((eeg_dataset[0]['window']))
#print(torch.min((no_normalize[0]['window'] - torch.min(no_normalize[0]['window']))/(torch.max(no_normalize[0]['window']))))

In [ ]:
# torch.save(model, 'oct7model.pt')